In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['df_2000_2001.csv.gz',
 'df_2000_2001_budget_revenue_title_cert.csv.gz',
 'df_clean_basics.csv.gz',
 'final_basics_df.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [ ]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)

In [ ]:
basics.head()

In [ ]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [ ]:
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [ ]:
ratings.head()

In [ ]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [ ]:
akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

In [ ]:
akas.head()

In [ ]:
basics = basics.replace({'\\N':np.nan})

In [ ]:
akas = akas.replace({'\\N':np.nan})

In [ ]:
ratings = ratings.replace({'\\N':np.nan})

In [ ]:
basics.info()

In [ ]:
basics['runtimeMinutes'].value_counts()

In [ ]:
basics.duplicated().sum()

In [ ]:
basics.isna().sum()

In [ ]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics.isna().sum()

In [ ]:
basics.dropna(subset=['genres'], inplace=True)
basics.isna().sum()

In [ ]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.isna().sum()

In [ ]:
basics['titleType'].value_counts()

In [ ]:
basics = basics[basics["titleType"].str.contains("movie")==True]
basics['titleType'].value_counts()

In [ ]:
basics['startYear'].value_counts()

In [ ]:
basics.dropna(subset=['startYear'], inplace=True)
basics.isna().sum()

In [ ]:
basics = basics[basics['startYear'] >= '2000']
basics['startYear'].value_counts()

In [ ]:
basics = basics[basics['startYear'] <= '2021']
basics['startYear'].value_counts()

In [ ]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

In [ ]:
basics = basics[keepers]
basics

In [ ]:
akas.head()

In [ ]:
akas['region'].value_counts()

In [ ]:
akas = akas[akas["region"].str.contains("US")==True]
akas['region'].value_counts()

In [ ]:
akas.info()

In [ ]:
basics.info()

In [ ]:
ratings.info()

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [ ]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [ ]:
import json
with open('C:/Users/fid24/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key-tmdb']

In [ ]:
YEARS_TO_GET = [2000,2001]

In [ ]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
 def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    # If it does not exist: create it
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:    
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # Load in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/title_basics.csv.gz')
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [ ]:
with open('C:/Users/fid24/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key-tmdb']

In [ ]:
JSON_FILE_2000 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2000.csv.gz"
JSON_FILE_2000

In [ ]:
df_2000 = pd.read_csv(JSON_FILE_2000)
df_2000.head()

In [ ]:
JSON_FILE_2001 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2001.csv.gz"
JSON_FILE_2001

In [ ]:
df_2001 = pd.read_csv(JSON_FILE_2001)
df_2001.head()

In [ ]:
df = pd.concat([df_2000, df_2001], ignore_index=True)
df

In [ ]:
df.info()

In [ ]:
df.to_csv("Data/df_2000_2001.csv.gz",compression='gzip',index=False)

In [ ]:
df.drop(columns='imdb_id', inplace=True)
df.drop(columns='adult', inplace=True)
df.drop(columns='backdrop_path', inplace=True)
df.drop(columns='belongs_to_collection', inplace=True)
df.drop(columns='homepage', inplace=True)
df.drop(columns='id', inplace=True)
df.drop(columns='original_language', inplace=True)
df.drop(columns='overview', inplace=True)
df.drop(columns='popularity', inplace=True)
df.drop(columns='poster_path', inplace=True)
df.drop(columns='production_companies', inplace=True)
df.drop(columns='production_countries', inplace=True)
df.drop(columns='release_date', inplace=True)
df.drop(columns='runtime', inplace=True)
df.drop(columns='spoken_languages', inplace=True)
df.drop(columns='status', inplace=True)
df.drop(columns='tagline', inplace=True)
df.drop(columns='video', inplace=True)
df.drop(columns='vote_average', inplace=True)
df.drop(columns='vote_count', inplace=True)
df.drop(columns='original_title', inplace=True)
df

In [ ]:
df.info()

In [ ]:
test_notebook = get_movie_with_rating("tt0332280")
test_notebook

In [ ]:
test_avengers = get_movie_with_rating("tt0848228")
test_avengers